In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-h87sessp
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-h87sessp
  Resolved https://github.com/facebookresearch/detectron2.git to commit 70f454304e1a38378200459dd2dbca0f0f4a5ab4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6174451 sha256=3ac7c00542c232b2b5f60c7ea617e

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
ls

sample_data/


In [3]:
cd drive/MyDrive/DeepLearning/panoptic/

/content/drive/MyDrive/DeepLearning/panoptic


In [ ]:
#cd drive/MyDrive/DeepLearning/panoptic/
#Kicina putanja

'/content/drive/MyDrive/DeepLearning/panoptic'

In [ ]:
import os
import random
import cv2
import matplotlib.pyplot as plt
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.data.datasets import register_coco_instances
from detectron2 import model_zoo
from google.colab import drive


# Korak 2: Registracija COCO anotacija za više datasetova
data_path_1 = './archive/Batch-1/Soccer/ann/instances.json'
data_path_2 = './archive/Batch-2/import-videos/ann/instances.json'
data_path_3 = './archive/Batch-3/import-videos/ann/instances.json'

register_coco_instances("my_dataset_train16", {}, data_path_1,  "archive/images/")
register_coco_instances("my_dataset_train17", {}, data_path_2,  "archive/images/")
register_coco_instances("my_dataset_train18", {}, data_path_3,  "archive/images/")

# Korak 3: Konfiguracija modela
cfg = get_cfg()
cfg.MODEL.DEVICE = "cuda"  # Use GPU

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train16", "my_dataset_train17", "my_dataset_train18")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 1  # Reduce batch size per image
cfg.SOLVER.BASE_LR = 0.001   # Reduce learning rate
cfg.SOLVER.MAX_ITER = 1000    # Increase number of iterations for better training
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64  # Reduce batch size per image
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 7  # Number of classes

# Korak 4: Treniranje modela
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[07/16 22:27:23 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:01, 160MB/s]
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[07/16 22:27:28 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[07/16 22:27:44 d2.utils.events]:  eta: 0:05:50  iter: 19  total_loss: 4.182  loss_cls: 2.023  loss_box_reg: 0.8821  loss_mask: 0.6928  loss_rpn_cls: 0.2107  loss_rpn_loc: 0.2283    time: 0.3574  last_time: 0.3538  data_time: 0.3056  last_data_time: 0.0200   lr: 1.9981e-05  max_mem: 1514M
[07/16 22:27:53 d2.utils.events]:  eta: 0:04:06  iter: 39  total_loss: 3.821  loss_cls: 1.736  loss_box_reg: 0.9173  loss_mask: 0.6805  loss_rpn_cls: 0.1946  loss_rpn_loc: 0.3173    time: 0.3006  last_time: 0.2596  data_time: 0.0121  last_data_time: 0.0009   lr: 3.9961e-05  max_mem: 1515M
[07/16 22:27:59 d2.utils.events]:  eta: 0:04:04  iter: 59  total_loss: 2.98  loss_cls: 1.176  loss_box_reg: 0.8663  loss_mask: 0.6508  loss_rpn_cls: 0.08908  loss_rpn_loc: 0.2594    time: 0.2991  last_time: 0.2574  data_time: 0.0468  last_data_time: 0.0007   lr: 5.9941e-05  max_mem: 1515M
[07/16 22:28:04 d2.utils.events]:  eta: 0:03:56  iter: 79  total_loss: 2.627  loss_cls: 0.7732  loss_box_reg: 0.8471  loss_mask: 0

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Korak 5: Postavi konfiguraciju za predikciju
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # Putanja do treniranog modela
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Prag za predikcije
predictor = DefaultPredictor(cfg)

# Funkcija za predikciju i čuvanje slika
def predict_and_save(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_name in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_name)
        im = cv2.imread(image_path)

        outputs = predictor(im)

        v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

        output_image_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_image_path, out.get_image()[:, :, ::-1])

# Putanje do ulaznog i izlaznog foldera PRAVI
input_folder = './archive/images_test/'
output_folder = './archive/predictions/'

# Primeni model na sve slike u folderu i sačuvaj rezultate
predict_and_save(input_folder, output_folder)



[07/16 22:36:20 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


In [ ]:
# Korak 5: Testiranje modela na novoj slici
# Postavi konfiguraciju za predikciju
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # Putanja do treniranog modela
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Prag za predikcije
predictor = DefaultPredictor(cfg)

# Učitaj test sliku
test_image_path = '../panoptic/archive/Batch-3/import-videos/images/proba.jpg'
im = cv2.imread(test_image_path)

# Napravi predikciju
outputs = predictor(im)

# Vizualizuj rezultate
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Prikaz slike sa predikcijama
plt.figure(figsize=(12, 12))
plt.imshow(out.get_image()[:, :, ::-1])
plt.axis('off')
plt.show()

[07/16 22:57:29 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


TypeError: Visualizer.draw_instance_predictions() got an unexpected keyword argument 'alpha'

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Funkcija za predikciju na jednoj slici
def predict_and_save_image(image_path, predictor, output_folder):
    # Učitaj sliku
    im = cv2.imread(image_path)

    # Napravi predikciju
    outputs = predictor(im)

    # Vizualizuj rezultate
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    # Kreiraj putanju za čuvanje rezultata
    output_image_path = os.path.join(output_folder, os.path.basename(image_path))

    # Sačuvaj rezultujuću sliku
    cv2.imwrite(output_image_path, out.get_image()[:, :, ::-1])

# Main funkcija za predikciju na folderu slika
def predict_on_folder(input_folder, output_folder, cfg):
    # Postavi konfiguraciju za predikciju
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # Putanja do treniranog modela
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Prag za predikcije
    predictor = DefaultPredictor(cfg)

    # Proveri da li izlazni folder postoji, ako ne, kreiraj ga
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iteriraj kroz sve slike u ulaznom folderu
    for image_name in os.listdir(input_folder):
        image_path = os.path.join(input_folder, image_name)
        if os.path.isfile(image_path):
            predict_and_save_image(image_path, predictor, output_folder)

# Konfiguracija modela
cfg = get_cfg()
cfg.merge_from_file("path/to/your/config/file.yaml")  # Zameni sa putanjom do tvoje konfiguracione datoteke

# Definiši ulazni i izlazni folder
input_folder = '../panoptic/archive/Batch-3/import-videos/images_test/'
output_folder = '../panoptic/archive/Batch-3/import-videos/predictions/'

# Pokreni predikciju na folderu slika
predict_on_folder(input_folder, output_folder, cfg)
